# Tensorflow Softmax

## Role of placeholder

The placeholder allows you to feed data into the computation graph by way of feed dictionaries. This gives you an entry point from your non-TensorFlow code in Python so you can run a graph on new inputs.